# Getting info on eSports teams from Liquipedia

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib

from time import sleep

In [18]:
continents_list = ['Americas', 'Europe', 'China']

In [13]:
teams_links = []

for cont in continents_list:
    res = requests.get('https://liquipedia.net/dota2/Portal:Teams/{cont}'.format(cont=cont))
    tree = BeautifulSoup(res.text, 'html5lib')
    
    for link in tree.find_all('span', class_='team-template-text'):
        teams_links.append(link.a['href'])

In [15]:
len(teams_links)

49

In [16]:
teams_links[0]

'/dota2/CompLexity_Gaming'

In [ ]:
for team in team_links:
    res = requests.get('https://liquipedia.net/{team}'.format(team=team))
    tree = BeautifulSoup()